## Baseline Model:

In [28]:
import pandas as pd
from sklearn import model_selection, preprocessing, metrics
import numpy as np
import datetime
import lightgbm as lgb
import matplotlib.pyplot as plt

train = pd.read_csv("test-flattened.csv", dtype={'fullVisitorId':str})#,parse_dates=['date'], date_parser=dateparse)
test = pd.read_csv("test-flattened.csv", dtype={'fullVisitorId':str})#,parse_dates=['date'], date_parser=dateparse)

train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,...,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions
0,Organic Search,20180511,7460955084541987166,1526099341,2,1526099341,Chrome,mobile,True,Android,...,3.0,NaN,(not set),(not set),True,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'APAC'}]"
1,Direct,20180511,460252456180441002,1526064483,166,1526064483,Chrome,desktop,False,Macintosh,...,3.0,NaN,(not set),(not set),True,(not set),(none),(not set),(direct),"[{'index': '4', 'value': 'North America'}]"
2,Organic Search,20180511,3461808543879602873,1526067157,2,1526067157,Chrome,desktop,False,Chrome OS,...,3.0,NaN,(not set),(not set),True,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'North America'}]"
3,Direct,20180511,975129477712150630,1526107551,4,1526107551,Chrome,mobile,True,iOS,...,4.0,NaN,(not set),(not set),True,(not set),(none),(not set),(direct),"[{'index': '4', 'value': 'North America'}]"
4,Organic Search,20180511,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet,True,Windows,...,4.0,NaN,(not set),(not set),NaN,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'North America'}]"


In [19]:
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,...,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions
0,Organic Search,2018-05-11,7460955084541987166,1526099341,2,1526099341,Chrome,mobile,True,Android,...,3.0,NaN,(not set),(not set),True,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'APAC'}]"
1,Direct,2018-05-11,460252456180441002,1526064483,166,1526064483,Chrome,desktop,False,Macintosh,...,3.0,NaN,(not set),(not set),True,(not set),(none),(not set),(direct),"[{'index': '4', 'value': 'North America'}]"
2,Organic Search,2018-05-11,3461808543879602873,1526067157,2,1526067157,Chrome,desktop,False,Chrome OS,...,3.0,NaN,(not set),(not set),True,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'North America'}]"
3,Direct,2018-05-11,975129477712150630,1526107551,4,1526107551,Chrome,mobile,True,iOS,...,4.0,NaN,(not set),(not set),True,(not set),(none),(not set),(direct),"[{'index': '4', 'value': 'North America'}]"
4,Organic Search,2018-05-11,8381672768065729990,1526060254,1,1526060254,Internet Explorer,tablet,True,Windows,...,4.0,NaN,(not set),(not set),NaN,(not provided),organic,(not set),google,"[{'index': '4', 'value': 'North America'}]"


In [20]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False)==1 ]
const_cols

[]

In [21]:
cols_to_drop = const_cols

train_df = train.drop(cols_to_drop, axis=1)
test_df = test.drop(cols_to_drop, axis=1)

In [22]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values

In [24]:
cat_cols = ['channelGrouping', 'device.browser', 'device.deviceCategory',
            'device.operatingSystem', 'geoNetwork.city',
            'geoNetwork.continent', 'geoNetwork.country', 
            'geoNetwork.metro','geoNetwork.networkDomain',
            'geoNetwork.region','geoNetwork.subContinent',
            'trafficSource.adContent', 'trafficSource.campaign',
            'trafficSource.isTrueDirect', 'trafficSource.keyword',
            'trafficSource.medium', 'trafficSource.referralPath',
            'trafficSource.source']

for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.campaign
trafficSource.isTrueDirect
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source


In [38]:
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 

dev_df = train_df[train_df['date']<=datetime.date(2018,6,30)]
val_df = train_df[train_df['date']>datetime.date(2018,6,30)]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)

dev_X = dev_df[cat_cols + num_cols] 
val_X = val_df[cat_cols + num_cols] 
test_X = test_df[cat_cols + num_cols] 

In [42]:
# custom function to run light gbm model
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 30,
        "min_child_samples" : 100,
        "learning_rate" : 0.1,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_val_y = model.predict(val_X, num_iteration=model.best_iteration)
    return pred_test_y, model, pred_val_y

# Training the model #
pred_test, model, pred_val = run_lgb(dev_X, dev_y, val_X, val_y, test_X)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.71398
Early stopping, best iteration is:
[92]	valid_0's rmse: 1.71222


In [43]:
# from sklearn import metrics

pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()

print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), 
                                         np.log1p(val_pred_df["PredictedRevenue"].values))))

1.8156875643139485


In [44]:
sub_df = pd.DataFrame({"fullVisitorId":test_id})
pred_test[pred_test<0] = 0
sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
sub_df.to_csv("baseline_lgb.csv", index=False)

In [46]:
sub_df.head(150)

,fullVisitorId,PredictedLogRevenue
0,0000018966949534117,0.015321
1,0000039738481224681,0.000000
2,0000073585230191399,0.000000
3,0000087588448856385,0.000000
4,0000149787903119437,0.000000
5,0000196310838896290,0.784903
6,00001995526696366,0.000000
7,0000255704530917106,0.456247
8,0000268499301061358,0.000000
9,0000276747989270229,0.006509
